DNN과 다르게 RNN에서는 okt형태소 토큰화 후 워드임베딩을 통해 문장의 문맥을 파악하는 방식으로 학습 시킬것이다. 
불용어를 제거=> 노이즈를 줄여서 성능향상을 도모

In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np


train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

train_text = open(train_file,'rb').read().decode(encoding='utf-8')



In [2]:
import pandas as pd

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [3]:
#데이터 크기 조절
df_train= df_train[::4]

In [4]:
#문장 추출
texts= [ ]
for i in df_train['feature']:
    texts.append(i)
len(texts)

37500

In [5]:
#문자열이 아닌게 있는지 확인
for i in texts:
    if type(i)!=str:
        print(i)

In [6]:
# train 데이터의 입력(X)에 대한 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)
    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

clean_texts=[]
for i in texts:
    clean_texts.append(clean_korean_text(i))

In [7]:
len(clean_texts)

30000

In [8]:
#문장을 단어단위로 토큰화해서 X값 만들기
texts= []
for i in clean_texts:
    print(i.split(' '))
    texts.append(i.split(' '))


['아', '더빙', '진짜', '짜증나네요', '목소리']
['막', '걸음마', '뗀', '세부터', '초등학교', '학년생인', '살용영화ㅋㅋㅋ별반개도', '아까움']
['걍인피니트가짱이다진짜짱이다']
['ㄱ냥', '매번', '긴장되고', '재밋음ㅠㅠ']
['나름', '심오한', '뜻도', '있는', '듯', '그냥', '학생이', '선생과', '놀아나는', '영화는', '절대', '아님']
['다', '짤랐을꺼야', '그래서', '납득할', '수', '없었던거야', '그럴꺼야', '꼭', '그랬던걸꺼야']
['엄포스의', '위력을', '다시', '한번', '깨닫게', '해준', '적남', '꽃검사님도', '연기', '정말', '좋았어요', '완전', '명품드라마']
['패션에', '대한', '열정', '안나', '윈투어']
['단순하면서', '은은한', '매력의', '영화']
['평점에속지마시길시간낭비', '돈낭비임']
['영화가', '사람의', '영혼을', '어루만져', '줄', '수도', '있군요', '거친', '세상사를', '잠시', '잊고', '동화같은', '영화에', '행복했네요']
['전', '좋아요']
['보는내내', '그대로', '들어맞는', '예측', '카리스마', '없는', '악역']
['많은', '사람들이', '이', '다큐를', '보고', '우리나라', '슬픈', '현대사의', '한', '단면에', '대해', '깊이', '생각하고', '사죄하고', '바로', '잡기', '위해', '노력했으면', '합니다', '말로만', '듣던', '보도연맹', '그', '민간인', '학살이', '이정도', '일', '줄이야', '이건', '명백한', '살인입니다', '살인자들은', '다', '어디있나요']
['아', '일본영화', '다이런건가', '유치하다']
['부패한', '로마노프', '왕조를', '기리는', '뭣같은', '영화', '온몸으로', '항거했던', '러시아', '민중들이', '그저', '폭도냐']
['뭐냐시작하고', '분만에', '

In [9]:
len(texts)

30000

In [10]:
#y값 추출
y_train=df_train.iloc[:,1]

#불균형데이터 확인
print(y_train[y_train==0]) #14973
print(y_train[y_train==1]) #15027


0         0
5         0
25        0
45        0
60        0
         ..
149940    0
149960    0
149985    0
149990    0
149995    0
Name: target, Length: 14973, dtype: int64
10        1
15        1
20        1
30        1
35        1
         ..
149955    1
149965    1
149970    1
149975    1
149980    1
Name: target, Length: 15027, dtype: int64


In [12]:
#타겟 라벨 데이터 차원 맞추기
y=y_train.to_frame()
print(y.shape)

#넘파이 배열로 변환
y=y.to_numpy()

(30000, 1)


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 텍스트 전처리
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)


In [15]:
#시퀀스데이터로 바꾸기
sequences = tokenizer.texts_to_sequences(texts)
x_train = pad_sequences(sequences, maxlen=30, padding='post')

In [16]:
#시퀀스 길이확인
print(x_train[0])
x_train.shape

[  34  818    4 4472 1225    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


(30000, 30)

In [23]:
# LSTM 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding,BatchNormalization,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Embedding( 5000, 300, input_length=30 ),
    LSTM(32),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# 모델 컴파일
optimizer= Adam(learning_rate =0.003)
model.compile(optimizer=optimizer , loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#콜백정의
ES =EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)

#모델학습
model.fit(x_train, y, epochs=20, batch_size=64, validation_split=0.25, callbacks=[ES])

Epoch 1/20
352/352 [==============================] - 23s 51ms/step - loss: 0.6288 - accuracy: 0.6112 - val_loss: 0.6988 - val_accuracy: 0.5643
Epoch 2/20
352/352 [==============================] - 18s 51ms/step - loss: 0.4769 - accuracy: 0.7603 - val_loss: 0.5143 - val_accuracy: 0.7380
Epoch 3/20
352/352 [==============================] - 17s 49ms/step - loss: 0.3990 - accuracy: 0.8017 - val_loss: 0.5144 - val_accuracy: 0.7453
Epoch 4/20
352/352 [==============================] - 17s 49ms/step - loss: 0.3497 - accuracy: 0.8231 - val_loss: 0.6294 - val_accuracy: 0.7089
Epoch 5/20
352/352 [==============================] - 17s 50ms/step - loss: 0.3189 - accuracy: 0.8361 - val_loss: 0.6844 - val_accuracy: 0.7345
Epoch 6/20
352/352 [==============================] - 18s 50ms/step - loss: 0.2898 - accuracy: 0.8511 - val_loss: 0.6361 - val_accuracy: 0.7329


In [24]:
#테스트문장 입력
test = '회식끝나고 집가는중'
test =test.split(' ')
test= [test]

sequences = tokenizer.texts_to_sequences(test) 
test_X_1 = pad_sequences(sequences,padding='post',maxlen=30) 
prediction = model.predict(test_X_1)  

if prediction[0][0] < prediction[0][1]:
    print('긍정문입니다.')
elif prediction[0][0] > prediction[0][1]:
    print('부정문입니다.')

1/1 [==============================] - 1s 1s/step
긍정문입니다.


In [ ]:
# model.save('기본 DNN 적용모델.h5')